### Importing and loading dataset - 21BAI1007

In [15]:
from keras.datasets import mnist
import numpy as np

In [16]:
(train_images, train_labels), (test_images, test_labels)= mnist.load_data()

In [17]:
train_images.shape, train_labels.shape

((60000, 28, 28), (60000,))

In [18]:
len(train_labels), len(test_labels)

(60000, 10000)

In [19]:
from keras import models
from keras import layers

In [20]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [21]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [22]:
features = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))

In [23]:
labels

array([5, 0, 4, ..., 4, 5, 6], dtype=uint8)

### Creating the model using K fold autotuning - 21BAI1007

In [25]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

for train, test in kfold.split(features, labels):

  model = Sequential()
  model.add(Dense(512, activation='relu', input_shape=(28*28,)))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

  model.fit(features[train], labels[train], epochs=10, batch_size=128, validation_data=(features[test], labels[test])) # evaluate the model
  scores = model.evaluate(features[test], labels[test], verbose=1)
  print("%s: %d" % (model.metrics_names[1], scores[1]))
  cvscores.append(scores[1] * 100)

Epoch 1/10
438/438 [==============================] - 2s 4ms/step - loss: 0.2749 - accuracy: 0.9227 - val_loss: 0.1358 - val_accuracy: 0.9603
Epoch 2/10
438/438 [==============================] - 1s 3ms/step - loss: 0.1114 - accuracy: 0.9674 - val_loss: 0.1020 - val_accuracy: 0.9699
Epoch 3/10
438/438 [==============================] - 1s 3ms/step - loss: 0.0726 - accuracy: 0.9788 - val_loss: 0.0787 - val_accuracy: 0.9754
Epoch 4/10
438/438 [==============================] - 1s 3ms/step - loss: 0.0511 - accuracy: 0.9846 - val_loss: 0.0649 - val_accuracy: 0.9794
Epoch 5/10
438/438 [==============================] - 1s 3ms/step - loss: 0.0382 - accuracy: 0.9892 - val_loss: 0.0606 - val_accuracy: 0.9811
Epoch 6/10
438/438 [==============================] - 2s 4ms/step - loss: 0.0277 - accuracy: 0.9926 - val_loss: 0.0596 - val_accuracy: 0.9815
Epoch 7/10
438/438 [==============================] - 2s 4ms/step - loss: 0.0209 - accuracy: 0.9946 - val_loss: 0.0649 - val_accuracy: 0.9794
Epoch 

### Creating the model using gridsearch - 21BAI1007

In [42]:
# pip install scikeras

# from keras.layers import Wrapper
# from scikeras.wrappers import KerasClassifier
# from sklearn.model_selection import GridSearchCV

# def create_model(optimizer="rmsprop", init = "glorot_uniform"):
#   model = Sequential()
#   model.add(Flatten(input_shape=(28,28)))
#   model.add(Dense(128, activation='relu'))
#   model.add(Dense(10, activation='softmax'))
#   model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
#   return model

# model2 = KerasClassifier(model=create_model, verbose=0)

# optimizers = ["rmsprop", "adam"]
# epochs = [50, 100]
# batches = [5, 10, 20]
# param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)

# grid = GridSearchCV(estimator=model2, param_grid=param_grid)
# grid_result = grid.fit(features, labels)

# ** Grid search is taking a lot of time to run, hence not including the output **

### Predicting the labels for the test images - 21BAI1007

In [33]:
pred = model.predict(features[test])

438/438 [==============================] - 1s 1ms/step


In [34]:
predicted_labels = np.argmax(pred, axis=1)
print(predicted_labels)

[3 5 3 ... 2 1 2]


In [36]:
labels[test]

array([3, 5, 3, ..., 2, 1, 2], dtype=uint8)

In [41]:
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(predicted_labels, labels[test])*100, "%")

Accuracy:  97.87857142857142 %
